In [ ]:
!pip install dwave-ocean-sdk

In [ ]:
import requests
import numpy as np
import pandas as pd
from dimod import BinaryQuadraticModel
from dimod import ExactSolver
from dwave.system import LeapHybridSampler
import sys, math

NUM_CURRENCIES = 4
LOOP_LENGTH = 4

# This list contains the currency codes. 
# It can contain more than NUM_CURRENCIES.
# All the exceeding entries will be discarded
CURRENCIES = ["USD", "EUR", "JPY", "LYD", "ZWL", "NGN", "FOK", "CAD"]

assert len(CURRENCIES) >= NUM_CURRENCIES, "At least NUM_CURRENCIES have to be specified."


In [ ]:
# Data retrieval

codes = CURRENCIES[:NUM_CURRENCIES]
matrix = np.array((NUM_CURRENCIES, NUM_CURRENCIES))
exchange_df = pd.DataFrame(columns=codes)

for code in codes:
  url = 'https://open.er-api.com/v6/latest/' + code
  response = requests.get(url)
  data = response.json()
  rates = data["rates"]
  df = pd.DataFrame(rates, index=[code,])
  df = df.filter(codes)
  exchange_df = exchange_df.append(df)

# Fix a profitable exchange rate
# exchange_df.loc['EUR', 'USD'] = 2
print(exchange_df)
  

         USD      EUR     JPY     LYD
USD        1  0.94700  127.74  4.8200
EUR     1.06  1.00000  134.90  5.0800
JPY  0.00783  0.00741    1.00  0.0377
LYD    0.208  0.19700   26.53  1.0000


In [ ]:
print(math.log(1/1.06))
print(math.log(1/0.94700))
print(math.log(1/0.0377))
print(math.log(1/26.53))

-0.05826890812397588
0.05445618579605896
3.278095184528172
-3.2782761681496484


In [ ]:
debug = False
class PositionalCurrency:

  def __init__(self, currency, position):
    self.currency = currency
    self.position = position
    self.id = self.currency + "x" + str(self.position)

  def __repr__(self):
    return self.id

def H_A(W, variable_i, variable_j):
  """
  This function returns a measure of the "arbitrage opportunity".
  A negative value means that profit is possible
  """
  if variable_j.position == ((variable_i.position + 1) % k):
    return W[variable_i.currency][variable_j.currency]
  elif variable_i.position == ((variable_j.position + 1) % k):
    return W[variable_j.currency][variable_i.currency]
  return 0

def H_B(variable_i, variable_j):
  """
  This function returns a positive value for PositionalCurrency pairs that 
  violate the following constraint:
    - two different currencies can not be in the same position
  """
  if variable_i.position == variable_j.position and variable_i.currency != variable_j.currency:
    return 1
  return 0


def H_E(variable_i, variable_j):
  """
  This function returns a reward (negative value) when any currency is present in any position.
  """
  if variable_i.position == variable_j.position and variable_i.currency == variable_j.currency:
    return -1
  return 0

# print(H_A(W, positional_currencies[0], positional_currencies[5]))

def H_C(variable_i, variable_j):
  if ((variable_i.position == 0 and variable_j.position == k-1) \
      or (variable_j.position == 0 and variable_i.position == k-1)) \
      and variable_i.currency == variable_j.currency:
    return 1
  return 0


def H_D(variable_i, variable_j):
  if (variable_i.position == variable_j.position + 1) \
      and variable_i.currency == variable_j.currency:
    return -1
  return 0


def H(W, variable_i, variable_j):

  ha = H_A(W, variable_i, variable_j)
  hb = H_B(variable_i, variable_j)
  hc = H_C(variable_i, variable_j)
  hd = H_D(variable_i, variable_j)
  he = H_E(variable_i, variable_j)

  #if variable_i.id == "EURx0" and variable_j.id == "USDx1":
  #  debug = True
  #else:
  #  debug = False

  if debug: 
    print("-----------------")
    print(variable_i.id, " -> ", variable_j.id)
    print("H_A:", A, "*", ha)
    print("H_B:", B, "*", hb)
    print("H_C:", C, "*", hc)
    print("H_D:", D, "*", hd)
    print("H_E:", B/k, "*", he)

  return A * H_A(W, variable_i, variable_j) + \
         B * H_B(variable_i, variable_j) + \
         B/k * H_E(variable_i, variable_j) + \
         D * H_D(variable_i, variable_j) + \
         C * H_C(variable_i, variable_j)

In [ ]:
list_1 = [1, 2, 4, 5, 6, 7, 8]
a = np.array(list_1)
a[[0, 2]]

array([1, 4])

In [ ]:
def inverse_log(x):
  return math.log(1/x)


k = LOOP_LENGTH

W = exchange_df.transpose().applymap(inverse_log)



positional_currencies = [PositionalCurrency(x, y) for x in codes for y in range(k)]
problem_set = [str(x) for x in positional_currencies]

def compute_sequence_profit(sample):
  ii = np.where(sample == 1)[0]
  loop_elements = np.array(positional_currencies)[ii]
  loop_elements = sorted(list(loop_elements), key=lambda x: x.position)
  profit = 1
  previous = None
  for element in loop_elements:
    if previous is not None:
      rate = exchange_df.loc[previous.currency, element.currency]
      profit = profit * rate
    previous = element
  return loop_elements, profit

# print(problem_set)

A = 1
B = 20
C = -7
D = -0.01 * exchange_df.to_numpy().min()

#H(W, positional_currencies[0], positional_currencies[LOOP_LENGTH+1])

num_variables = LOOP_LENGTH * NUM_CURRENCIES
# bqm = np.zeros((num_variables, num_variables))


bqm = BinaryQuadraticModel(num_variables, 'BINARY')

print("\t", *[ pc.id + "\t" for pc in positional_currencies], sep="")
for i, pos_currency_i in enumerate(positional_currencies):
  print(pos_currency_i.id, end="\t")
  for j, pos_currency_j in enumerate(positional_currencies):
    interaction = H(W, pos_currency_i, pos_currency_j)
    print("{0:.3g}".format(interaction), end="\t")
    if i == j: 
      bqm.add_linear(i, interaction)
      break
    else:
      bqm.add_interaction(i, j, interaction)
    
  print()


sampler = ExactSolver()

sampleset = sampler.sample(bqm, label='Currency arbitrage')
sample = sampleset.first

sorted_records = sorted(sampleset.record, key=lambda x: x["energy"] )

for rec in sorted_records[:25]:
  print(rec, compute_sequence_profit(rec.sample))

#print(sampleset.record)
print("---------")
print(sample)







	USDx0	USDx1	USDx2	USDx3	EURx0	EURx1	EURx2	EURx3	JPYx0	JPYx1	JPYx2	JPYx3	LYDx0	LYDx1	LYDx2	LYDx3	
USDx0	-5	
USDx1	7.41e-05	-5	
USDx2	0	7.41e-05	-5	
USDx3	-7	0	7.41e-05	-5	
EURx0	20	-0.0583	0	0.0545	-5	
EURx1	0.0545	20	-0.0583	0	7.41e-05	-5	
EURx2	0	0.0545	20	-0.0583	0	7.41e-05	-5	
EURx3	-0.0583	0	0.0545	20	-7	0	7.41e-05	-5	
JPYx0	20	4.85	0	-4.85	20	4.9	0	-4.9	-5	
JPYx1	-4.85	20	4.85	0	-4.9	20	4.9	0	7.41e-05	-5	
JPYx2	0	-4.85	20	4.85	0	-4.9	20	4.9	0	7.41e-05	-5	
JPYx3	4.85	0	-4.85	20	4.9	0	-4.9	20	-7	0	7.41e-05	-5	
LYDx0	20	1.57	0	-1.57	20	1.62	0	-1.63	20	-3.28	0	3.28	-5	
LYDx1	-1.57	20	1.57	0	-1.63	20	1.62	0	3.28	20	-3.28	0	7.41e-05	-5	
LYDx2	0	-1.57	20	1.57	0	-1.63	20	1.62	0	3.28	20	-3.28	0	7.41e-05	-5	
LYDx3	1.57	0	-1.57	20	1.62	0	-1.63	20	-3.28	0	3.28	20	-7	0	7.41e-05	-5	


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: SamplerUnknownArgWarning: Ignoring unknown kwarg: 'label'


([0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0], -27.00649129, 1) ([EURx0, USDx1, LYDx2, EURx3], 1.0065124)
([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], -27.00649129, 1) ([LYDx0, EURx1, USDx2, LYDx3], 1.0065124)
([1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], -27.00649129, 1) ([USDx0, LYDx1, EURx2, USDx3], 1.0065124)
([1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -27.00373862, 1) ([USDx0, EURx1, USDx2, USDx3], 1.00382)
([1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], -27.00373862, 1) ([USDx0, EURx1, EURx2, USDx3], 1.00382)
([1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], -27.00373862, 1) ([USDx0, USDx1, EURx2, USDx3], 1.00382)
([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], -27.00373862, 1) ([EURx0, USDx1, EURx2, EURx3], 1.00382)
([0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], -27.00373862, 1) ([EURx0, EURx1, USDx2, EURx3], 1.00382)
([0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], -27.00373862, 1) ([EURx0, USDx1, USDx2, EURx3], 1.00382)
([0, 1, 0, 0, 1, 0, 0,

In [ ]:
from dimod import BinaryQuadraticModel

def exact_cover_bqm(problem_set, subsets):  
    """Returns a BQM for an exact cover.
 
    An exact cover is a collection of subsets of `problem_set` that contains every 
    element in `problem_set` exactly once.
    
    Args:
        problem_set : iterable
            An iterable of unique numbers.

        subsets : list(iterable(numeric))
            A list of subsets of `problem_set` used to find an exact cover.
    """
    bqm = BinaryQuadraticModel({}, {}, 0, 'BINARY')

    for element in problem_set: # consideriamo solo righe e colonne perché ci fermiamo a 2n
        bqm.offset += 1

        for i in range(len(subsets)):    # subset = lista di elementi con  4 vincoli ciascuno
            if element in subsets[i]:  # element = una cella (id)
                bqm.add_variable(i, -1) 

                for j in range(i):         
                    if element in subsets[j]:
                        bqm.add_interaction(i, j, 2)  # costo, perché vanno in conflitto i vincoli di riga o colonna

    return bqm

In [ ]:
# Copyright 2020 D-Wave Systems Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from collections import Counter

import numpy as np
import matplotlib
matplotlib.use("agg")    # must select backend before importing pyplot
import matplotlib.pyplot as plt
from dimod import BinaryQuadraticModel
from dimod import ExactSolver
from dwave.system import LeapHybridSampler

def build_subsets(n):
    """Returns a list of subsets of constraints corresponding to every
    position on the chessboard.

    Each constraint is represented by a unique number (ID). Each subset
    should contain:
    1) Exactly one column constraint ID (0 to n-1).
    2) Exactly one row constraint ID (n to 2*n-1).
    3) At most one diagonal (top-left to bottom-right) constraint ID (2*n to
       4*n-4).
    4) At most one anti-diagonal (bottom-left to top-right) constraint ID (4*n-3
       to 6*n-7).
    """
    subsets = []
    for x in range(n):
        for y in range(n):
            col = x
            row = y + n

            subset = {col, row}

            diag = x + y + (2*n - 1)
            min_diag = 2*n
            max_diag = 4*n - 4

            if diag >= min_diag and diag <= max_diag:
                subset.add(diag)

            anti_diag = (n - 1 - x + y) + (4*n - 4)
            min_anti_diag = 4*n - 3
            max_anti_diag = 6*n - 7

            if anti_diag >= min_anti_diag and anti_diag <= max_anti_diag:
                subset.add(anti_diag)

            subsets.append(subset)

    return subsets

def handle_diag_constraints(bqm, subsets, diag_constraints):
    """Update bqm with diagonal (and anti-diagonal) constraints.
    Duplicates are penalized.
    """
    for constraint in diag_constraints:
        for i in range(len(subsets)):
            if constraint in subsets[i]:
                for j in range(i):
                    if constraint in subsets[j]:
                        bqm.add_interaction(i, j, 2)
    return bqm

def n_queens(n, sampler=None):
    """Returns a potential solution to the n-queens problem in a list of sets,
    each containing constraint IDs representing a queen's location.

    Args:
        n: Number of queens to place.

        sampler: A binary quadratic model sampler. Defaults to dwave-system's LeapHybridSampler.
    """
    num_row_col_constraints = 2 * n
    row_col_constraint_ids = set(range(num_row_col_constraints))

    num_diag_constraints = 4 * n - 6   # includes anti-diag constraints
    diag_constraint_ids = set(range(num_row_col_constraints, num_row_col_constraints + num_diag_constraints))

    # Build subsets of constraint IDs. Each subset will become a variable in our BQM.
    subsets = build_subsets(n)

    # Build BQM with only row/col constraints
    bqm = exact_cover_bqm(row_col_constraint_ids, subsets)

    # Add diag/anti-diag constraints - duplicates are penalized.
    bqm = handle_diag_constraints(bqm, subsets, diag_constraint_ids)

    if sampler is None:
        sampler = ExactSolver()

    sampleset = sampler.sample(bqm, label='Example - N Queens')
    sample = sampleset.first.sample

    return [subsets[i] for i in sample if sample[i]]

def is_valid_solution(n, solution):
    """Check that solution is valid by making sure all constraints were met.

    Args:
        n: Number of queens in the problem.

        solution: A list of sets, each containing constraint IDs that represent
                  a queen's location.
    """
    count = Counter()

    for queen in solution:
        count = count + Counter(queen)

    # Check row/col constraints
    for i in range(2*n):
        if count[i] != 1:
            if i < n:
                col = i
                print("Column {} has {} queens.".format(col, count[i]))
            else:
                row = np.abs(i - (2*n - 1)) # Convert constraint id to row index
                print("Row {} has {} queens.".format(row, count[i]))

            return False

    # Check diag/anti-diag constraints
    for i in range(2*n, 4*n - 6):
        if count[i] > 1:
            if i <= 4*n - 4:
                print("Top-left to bottom-right diagonal {} has {} queens.".format(i, count[i]))
            else:
                print("Bottom-left to top-right diagonal {} has {} queens.".format(i, count[i]))

            return False

    return True

def plot_chessboard(n, queens):
    """Create a chessboard with queens using matplotlib. Image is saved
    in the root directory. Returns the image file name.
    """
    chessboard = np.zeros((n,n))
    chessboard[1::2,0::2] = 1
    chessboard[0::2,1::2] = 1

    # Adjust fontsize for readability
    if n <= 10:
        fontsize = 30
    elif n <= 20:
        fontsize = 10
    else:
        fontsize = 5

    plt.xticks(np.arange(n))
    plt.yticks(np.arange(n))

    plt.imshow(chessboard, cmap='binary')

    # Place queens
    for subset in solution:
        x = y = -1
        for constraint in subset:
            if constraint < n:
                x = constraint
            elif constraint >= n and constraint < 2*n:
                y = np.abs(constraint - (2*n - 1)) # Convert constraint ID to row index

        if x != -1 and y != -1:
            plt.text(x, y, u"\u2655", fontsize=fontsize, ha='center',
                     va='center', color='black' if (x - y) % 2 == 0 else 'white')

    # Save file in root directory
    file_name = "{}-queens-solution.png".format(n)
    plt.savefig(file_name)

    return file_name

def get_sanitized_input():
    while True:
        print("Enter the number of queens to place (n > 0):")
        n = input()

        try:
            n = int(n)
            if n <= 0:
                print("Input must be greater than 0.")
                continue
            if n >= 200:
                # Run but give a warning
                print("Problems with large n will run very slowly.")

        except ValueError:
            print("Input type must be int.")
            continue

        return n

if __name__ == "__main__":
    n = get_sanitized_input()

    if n > 20:
        print("Solution image is large and may be difficult to view.")
        print("Plot settings in plot_chessboard() may need adjusting.")

    print("Trying to place {n} queens on a {n}*{n} chessboard.".format(n=n))
    solution = n_queens(n)

    if is_valid_solution(n, solution):
        print("Solution is valid.")
    else:
        print("Solution is invalid.")

    file_name = plot_chessboard(n, solution)
    print("Chessboard created. See: {}".format(file_name))



Enter the number of queens to place (n > 0):


KeyboardInterrupt: ignored